In [1]:
!pip install altair
!pip install altair
!pip install nltk
!pip install tensorflow

import numpy as np
import pandas as pd
import altair as alt
from tqdm import tqdm
import time
import re
from scipy.sparse import hstack

import nltk
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, KFold
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import tensorflow as tf
import keras
from keras import layers
from keras.layers import TextVectorization

     |████████████████████████████████| 813 kB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 12.0 MB/s  eta 0:00:01
     |████████████████████████████████| 1.5 MB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 772 kB 126.6 MB/s eta 0:00:01
     |████████████████████████████████| 588.3 MB 15 kB/s /s eta 0:00:01
     |████████████████████████████████| 1.7 MB 117.6 MB/s eta 0:00:01
     |████████████████████████████████| 21.5 MB 135.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.8 MB 128.5 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 13.7 MB/s  eta 0:00:01
     |████████████████████████████████| 4.7 MB 129.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 119.6 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 27.8 MB/s  eta 0:00:01
     |████████████████████████████████| 126 kB 138.5 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 10.4 MB/s  eta 0:00:01
     |███████████████████

2023-02-27 17:47:34.407003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 17:47:34.523142: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/oracle/instantclient_12_1:/usr/lib/jvm/java-11-openjdk-amd64/jre/lib/amd64/server:
2023-02-27 17:47:34.523175: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-27 17:47:35.124492: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic li

In [2]:
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Data Preparation

# Binary/ Count/ TFIDF Vectorization

## 1) Import New Dataset

In [3]:
# Load data from CSV file
new_data = pd.read_csv('../results/new_text_data.csv')

# Drop all rows that contain Null Entries in 'lemma_text' column
new_data = new_data.dropna(subset=['lemma_text'])

## 2) Binary/Count/TFIDF Vectorization

In [4]:
# Function to identify all columns that are not 'label' in a DataFrame
def determine_X_feat(df):
    all_features = df.columns.to_list()
    X_feat = []
    for feat in all_features:
        if feat != 'label':
            X_feat.append(feat)
    return X_feat

In [5]:
def scikit_column_transformer(text_df = new_data, text_type = 'original_text', vector_type = 'Count',
                              scaler='Robust', ngrams_value=1, max_features_value=None, sequence_length=500, 
                              test_size=0.2, random_state=21):
  
    # Reduce the input dataframe to only include either the original_text or lemma_text columns
    if text_type == 'original_text':
        final_text_df = text_df.drop(columns=['lemma_text'])
        final_text_df = final_text_df.rename(columns={'original_text': 'text'})
        
    elif text_type == 'lemma_text':
        final_text_df = text_df.drop(columns=['original_text'])
        final_text_df = final_text_df.rename(columns={'lemma_text': 'text'})
        
    else:
        return 'Incorrect input for text_type argument'


    # Perform the Train-Test Split Based on Input Data
    X_feat = determine_X_feat(final_text_df)
    X_train, X_test, y_train, y_test = train_test_split(final_text_df[X_feat], final_text_df['label'],
                                                      test_size=test_size, random_state=random_state)


    # Select Vectors for text data and POS data
    if vector_type == 'Count':
        text_vector = CountVectorizer(ngram_range=(1, ngrams_value),max_features=max_features_value)
        pos_vector = CountVectorizer(ngram_range=(1, ngrams_value), preprocessor=None,
                                     token_pattern=r'[^\s]+', lowercase=False)
    elif vector_type == 'Tfidf':
        text_vector = TfidfVectorizer(ngram_range=(1, ngrams_value), max_features=max_features_value)
        pos_vector = TfidfVectorizer(ngram_range=(1, ngrams_value),
                                     token_pattern=r'[^\s]+', lowercase=False)
    elif vector_type == 'Binary':
        text_vector = CountVectorizer(binary=True, ngram_range=(1, ngrams_value),max_features=max_features_value)
        pos_vector = CountVectorizer(binary=True, ngram_range=(1, ngrams_value), preprocessor=None,
                                     token_pattern=r'[^\s]+', lowercase=False)
    elif vector_type == 'Indice':
        text_vector = TextVectorization(output_mode="int", max_tokens=max_features_value,
                                   output_sequence_length=sequence_length, ngrams=(1, ngrams_value))
    else:
        return 'Incorrect input for vector_type argument'


    # Select the desired scaler based on input string
    dict_of_scalers = {'Robust': RobustScaler(), 'MinMax': MinMaxScaler(),
                       'Standard': StandardScaler(), 'drop': 'drop'}
    
    try:
        selected_feature_scaler = dict_of_scalers[scaler]
    except:
        return 'Incorrect input for scaler argument - must be either Count, MinMax, Standard or drop'


    # Use Scikit-Learn Column Transformer to vectorize the text data and the POS data,
    # and transform the additional features by selected scaler
    if vector_type != 'Indice':
        column_trans = ColumnTransformer([('vector_text', text_vector, 'text'), 
                                        ('vector_pos_tags', pos_vector, 'pos_tag_tokens')], 
                                      remainder = selected_feature_scaler)

        # Perform Fit_Transform on X_train and transform on X_test  
        X_train_matrix = column_trans.fit_transform(X_train)
        X_test_matrix = column_trans.transform(X_test)

    else:
        column_trans = None

        # Perform Fit_Transform on X_train and transform on X_test
        text_vector.adapt(X_train['text'])
        X_train_matrix = text_vector(X_train['text'])
        X_test_matrix = text_vector(X_test['text'])

    return column_trans, X_train_matrix, y_train, X_test_matrix, y_test

In [6]:
# Generate Vectorized and Scaled DataFrames for Supervised Learning
tfidf_trans, X_train_tfidf, y_train_tfidf, X_test_tfidf, y_test_tfidf =\
    scikit_column_transformer(text_df=new_data, text_type='lemma_text',max_features_value=40000,
                              vector_type='Tfidf', scaler='Robust')

binary_trans, X_train_bin, y_train_bin, X_test_bin, y_test_bin =\
    scikit_column_transformer(text_df=new_data, text_type='lemma_text', max_features_value=40000,
                              vector_type='Binary', scaler='Robust')

count_trans, X_train_count, y_train_count, X_test_count, y_test_count =\
    scikit_column_transformer(text_df=new_data, text_type='lemma_text', max_features_value=40000,
                              vector_type='Count', scaler='MinMax')

# Indice Vectorization

## 1) Import original dataset

In [7]:
old_data_1 = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/WikiLarge_Train_part_1.csv')
old_data_2 = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/WikiLarge_Train_part_2.csv')
old_data_3 = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/WikiLarge_Train_part_3.csv')
old_data = pd.concat([old_data_1, old_data_2, old_data_3], ignore_index=True)

X_train, X_test, y_train_indice, y_test_indice = \
    train_test_split(old_data['original_text'], old_data['label'], test_size=0.2, random_state=21)

## 2) Indice Vectorization

In [8]:
# refer https://towardsdatascience.com/predicting-the-difficulty-of-texts-using-machine-learning-and-getting-a-visual-representation-of-75f5a96b92e5

def preprocessing(text_series):
    text_list = []
    lemmatizer = WordNetLemmatizer()
    
    for text in tqdm(text_series.values):
        text = re.sub('[^a-zA-Z]|LRB|RRB', ' ', text)
        text = text.lower()
        text = nltk.word_tokenize(text)
        #text = [word for word in text if not word in set(stopwords.words("english"))]
        text = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(text)
        
        text_list.append(text)
    
    return text_list

In [9]:
def indVec(trainText_series, testText_series, ngrams=1, max_features=40000, sequence_length=500):
    trainTextPreprocessed = preprocessing(trainText_series)
    testTextPreprocessed = preprocessing(testText_series)
    
    vector = TextVectorization(output_mode="int", max_tokens=max_features,
                               output_sequence_length=sequence_length, ngrams=(1, ngrams))
    vector.adapt(trainTextPreprocessed)
    trainText_matrix = vector(trainTextPreprocessed)
    testText_matrix = vector(testTextPreprocessed)
    
    return (trainText_matrix, testText_matrix)

In [10]:
X_train_indice, X_test_indice = indVec(X_train, X_test)

100%|██████████| 83354/83354 [00:13<00:00, 6364.83it/s]
2023-02-27 17:49:25.579633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/oracle/instantclient_12_1:/usr/lib/jvm/java-11-openjdk-amd64/jre/lib/amd64/server:
2023-02-27 17:49:25.579677: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-27 17:49:25.579717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (run-63fceaf10fc863061e25bcc6-v9z6f): /proc/driver/nvidia/version does not exist
2023-02-27 17:49:25.579999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical o

# Supervised Learning

## 1) Naive Bayes

### a) Bernoulli

In [11]:
start_time = time.time()

# train Bernoulli model
# optimize the hyperparameters
res = pd.DataFrame(columns=['alpha', 'trainScore', 'testScore'])

for alpha in tqdm([0.01, 0.1, 1, 10]):
    clf = BernoulliNB(alpha=alpha)
        
    clf.fit(X_train_bin, y_train_bin)
    trainScore = clf.score(X_train_bin, y_train_bin)
    testScore = clf.score(X_test_bin, y_test_bin)

    res = res.append({'alpha':alpha, 'trainScore':trainScore, 'testScore':testScore},
                    ignore_index=True)

# get run time
end_time = time.time()
delta_time = end_time - start_time
print('\n')
print("total run time: {} mins".format(delta_time/60))

100%|██████████| 4/4 [00:01<00:00,  2.18it/s]



total run time: 0.030609321594238282 mins


In [12]:
# plot the hyperparameters search result

resLong = res.melt(id_vars=['alpha'], value_vars=['trainScore', 'testScore'],
         var_name='group', value_name='score')

Lineplot = alt.Chart(resLong).mark_line().encode(
    x = 'alpha:N',
    y = alt.Y('score', scale=alt.Scale(domain=[0.5,1])),
    color = 'group',
    tooltip = 'score'
).properties(width=100, height=100)

Lineplot

alt.Chart(...)

In [13]:
# generate report result
ber = BernoulliNB(alpha=1)
ber_re = cross_validate(clf, X_train_bin, y_train_bin, cv=5, scoring=['accuracy', 'recall'])

In [14]:
pd.DataFrame(ber_re).describe()

,fit_time,score_time,test_accuracy,test_recall
count,5.000000,5.000000,5.000000,5.000000
mean,0.201058,0.054039,0.634084,0.605773
std,0.000936,0.001264,0.002080,0.002484
min,0.199986,0.053243,0.630864,0.602244
25%,0.200522,0.053445,0.633086,0.604090
50%,0.200917,0.053516,0.635381,0.607080
75%,0.201421,0.053713,0.635508,0.607564
max,0.202443,0.056280,0.635580,0.607890


### b) Multinominal

In [15]:
start_time = time.time()

# train Multinominal model
res = pd.DataFrame(columns=['alpha', 'trainScore', 'testScore'])

for alpha in tqdm([0.1, 1, 10, 100]):
    clf = MultinomialNB(alpha=alpha)

    clf.fit(X_train_count, y_train_count)
    trainScore = clf.score(X_train_count, y_train_count)
    testScore = clf.score(X_test_count, y_test_count)

    res = res.append({'alpha':alpha, 'trainScore':trainScore, 'testScore':testScore},
                    ignore_index=True)

# get run time
end_time = time.time()
delta_time = end_time - start_time
print('\n')
print("total run time: {} mins".format(delta_time/60))

100%|██████████| 4/4 [00:00<00:00,  5.94it/s]



total run time: 0.011260414123535156 mins


In [16]:
resLong = res.melt(id_vars=['alpha'], value_vars=['trainScore', 'testScore'],
         var_name='group', value_name='score')

Lineplot = alt.Chart(resLong).mark_line().encode(
    x = 'alpha:N',
    y = alt.Y('score', scale=alt.Scale(domain=[0.5,1])),
    color = 'group',
    tooltip = 'score'
).properties(width=100, height=100)

Lineplot

alt.Chart(...)

In [17]:
# generate report result
mul = MultinomialNB(alpha=10)
mul_re = cross_validate(mul, X_train_count, y_train_count, cv=5, scoring=['accuracy', 'recall'])

In [18]:
pd.DataFrame(mul_re).describe()

,fit_time,score_time,test_accuracy,test_recall
count,5.000000,5.000000,5.000000,5.000000
mean,0.100820,0.029361,0.656147,0.696581
std,0.000360,0.000263,0.002467,0.002180
min,0.100377,0.029025,0.652891,0.694282
25%,0.100632,0.029227,0.655240,0.695765
50%,0.100737,0.029300,0.655764,0.696164
75%,0.101062,0.029610,0.657318,0.696526
max,0.101292,0.029646,0.659523,0.700170


## 2) Logistic Regression

In [19]:
start_time = time.time()

# train logistic regression model
res = pd.DataFrame(columns=['C', 'trainScore', 'testScore'])

for C in tqdm([0.01, 0.1, 1, 10]):
    clf = LogisticRegression(C=C, solver='lbfgs', n_jobs=-1,
                              random_state=0, max_iter=1000)
    
    clf.fit(X_train_tfidf, y_train_tfidf)
    trainScore = clf.score(X_train_tfidf, y_train_tfidf)
    testScore = clf.score(X_test_tfidf, y_test_tfidf)

    res = res.append({'C':C, 'trainScore':trainScore, 'testScore':testScore},
                ignore_index=True)

# get run time
end_time = time.time()
delta_time = end_time - start_time
print('\n')
print("total run time: {} mins".format(delta_time/60))

100%|██████████| 4/4 [02:36<00:00, 39.19s/it]



total run time: 2.6129480679829915 mins


In [20]:
resLong = res.melt(id_vars=['C'], value_vars=['trainScore', 'testScore'],
         var_name='group', value_name='score')

Lineplot = alt.Chart(resLong).mark_line().encode(
    x = 'C:N',
    y = alt.Y('score', scale=alt.Scale(domain=[0.5,1])),
    color = 'group',
    tooltip = 'score'
).properties(width=100, height=100)

Lineplot

alt.Chart(...)

In [21]:
# generate report result
logRe = LogisticRegression(C=0.1, solver='lbfgs', max_iter=1500)
logRe_re = cross_validate(logRe, X_train_tfidf, y_train_tfidf, cv=5, scoring=['accuracy', 'recall'])

In [22]:
pd.DataFrame(logRe_re).describe()

,fit_time,score_time,test_accuracy,test_recall
count,5.000000,5.000000,5.000000,5.000000
mean,29.887921,0.038750,0.685963,0.688496
std,2.443775,0.003256,0.002444,0.004504
min,25.727132,0.036759,0.682364,0.680890
25%,29.876733,0.036906,0.684930,0.688237
50%,30.917706,0.037333,0.686357,0.690156
75%,30.921098,0.038276,0.687477,0.690807
max,31.996933,0.044478,0.688688,0.692388


## 3) Ensembles

### a) Random Forest

In [23]:
start_time = time.time()

# train random forest model
res = pd.DataFrame(columns=['n_estimators', 'max_depth', 'trainScore', 'testScore'])


for n_estimators in tqdm([10, 100, 200]):
    for max_depth in tqdm([5, 50, 100, 250]):
        clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                     n_jobs=-1, random_state=0)
        
        clf.fit(X_train_tfidf, y_train_tfidf)
        trainScore = clf.score(X_train_tfidf, y_train_tfidf)
        testScore = clf.score(X_test_tfidf, y_test_tfidf)

        res = res.append({'n_estimators':n_estimators, 'max_depth':max_depth,
                          'trainScore':trainScore, 'testScore':testScore},
                          ignore_index=True)

# get run time
end_time = time.time()
delta_time = end_time - start_time
print('\n')
print("total run time: {} mins".format(delta_time/60))

 50%|█████     | 2/4 [00:26<00:31, 15.51s/it]/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mod



total run time: 14.16754190524419 mins


In [25]:
Heatmap = alt.Chart(res).mark_rect(stroke='white').encode(
    x = 'n_estimators:N',
    y = 'max_depth:N',
    color = 'testScore',
    tooltip = 'testScore'
).properties(width=100, height=100)


resLong = res.melt(id_vars=['n_estimators', 'max_depth'], value_vars=['trainScore', 'testScore'],
         var_name='group', value_name='score')

Lineplot = alt.Chart(resLong).mark_line().encode(
    x = 'n_estimators:N',
    y = alt.Y('score', scale=alt.Scale(domain=[0.5,1])),
    color = 'group',
    column = 'max_depth'
).properties(width=100, height=100)

(Heatmap | Lineplot)

alt.HConcatChart(...)

In [26]:
# generate report result
randFr = RandomForestClassifier(n_estimators=100, max_depth=50,
                                     n_jobs=-1, random_state=0)
randFr_re = cross_validate(randFr, X_train_tfidf, y_train_tfidf, cv=5, scoring=['accuracy', 'recall'])

In [27]:
pd.DataFrame(randFr_re).describe()

,fit_time,score_time,test_accuracy,test_recall
count,5.000000,5.000000,5.000000,5.000000
mean,19.031434,0.136392,0.673509,0.712636
std,1.097752,0.003268,0.002600,0.005767
min,18.315338,0.133120,0.669443,0.703004
25%,18.331895,0.133252,0.672588,0.712704
50%,18.730489,0.137272,0.674286,0.713355
75%,18.828076,0.137412,0.675298,0.716504
max,20.951374,0.140903,0.675931,0.717616


### b) Gradient Boosting

In [28]:
start_time = time.time()

# train Gradient Boosting model
res = pd.DataFrame(columns=['n_estimators', 'leraning_rate', 'trainScore', 'testScore'])

for n_estimators in tqdm([10, 100, 200]):
    for learning_rate in tqdm([0.01, 0.1, 1, 10]):
        clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                                     random_state=0)
        
        clf.fit(X_train_tfidf, y_train_tfidf)
        trainScore = clf.score(X_train_tfidf, y_train_tfidf)
        testScore = clf.score(X_test_tfidf, y_test_tfidf)

        res = res.append({'n_estimators':n_estimators, 'learning_rate':learning_rate,
                          'trainScore':trainScore, 'testScore':testScore},
                          ignore_index=True)

# get run time
end_time = time.time()
delta_time = end_time - start_time
print('\n')
print("total run time: {} mins".format(delta_time/60))

100%|██████████| 3/3 [1:11:09<00:00, 1423.29s/it]



total run time: 71.16457213163376 mins


In [29]:
Heatmap = alt.Chart(res).mark_rect(stroke='white').encode(
    x = 'n_estimators:N',
    y = 'learning_rate:N',
    color = 'testScore',
    tooltip = 'testScore'
).properties(width=100, height=100)


resLong = res.melt(id_vars=['n_estimators', 'learning_rate'], value_vars=['trainScore', 'testScore'],
         var_name='group', value_name='score')

Lineplot = alt.Chart(resLong).mark_line().encode(
    x = 'n_estimators:N',
    y = alt.Y('score', scale=alt.Scale(domain=[0.5,1])),
    color = 'group',
    column = 'learning_rate'
).properties(width=100, height=100)

(Heatmap | Lineplot)

alt.HConcatChart(...)

In [30]:
# generate report result
gradB = GradientBoostingClassifier(n_estimators=100, learning_rate=1,
                                     random_state=0)
gradB_re = cross_validate(gradB, X_train_tfidf, y_train_tfidf, cv=5, scoring=['accuracy', 'recall'])

In [31]:
pd.DataFrame(gradB_re).describe()

,fit_time,score_time,test_accuracy,test_recall
count,5.000000,5.000000,5.000000,5.000000
mean,274.188533,0.110713,0.695201,0.712303
std,0.456321,0.002288,0.001687,0.002094
min,273.838341,0.108605,0.692609,0.709917
25%,273.865618,0.108909,0.694850,0.711509
50%,274.050493,0.110570,0.695175,0.711799
75%,274.234958,0.111163,0.696422,0.712704
max,274.953253,0.114320,0.696946,0.715589


## 4) Deep Learning

### a) FFNN

In [32]:
X_train_tfidf.sort_indices()
X_test_tfidf.sort_indices()

In [33]:
start_time = time.time()

layers1 = [20, 100, 200, 500]
layers2 = [20, 50, 100]
layers3 = [5, 20]
input_shape = X_train_tfidf.shape[1]
ep_size = 3
batch_size = 128

res_FFNN = pd.DataFrame(columns=['layer1', 'layer2', 'layer3', 'epoch', 'trainScore', 'testScore'])

for units_1 in tqdm(layers1):
    for units_2 in tqdm(layers2):
        for units_3 in tqdm(layers3):
            
            inputs = keras.Input(shape=(input_shape, ))
            x = layers.Dense(units_1, activation='relu')(inputs)
            x = layers.Dense(units_2, activation='relu')(x)
            x = layers.Dense(units_3, activation='relu')(x)
            outputs = layers.Dense(1, activation='sigmoid')(x)

            model = keras.Model(inputs=inputs, outputs=outputs)
            model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
            model.fit(x=X_train_tfidf, y=y_train_tfidf.values, validation_data=(X_test_tfidf, y_test_tfidf.values),
                epochs=ep_size, batch_size=batch_size, verbose=1)
            
            history = model.history.history
            
            for ep in np.arange(ep_size):
                res_FFNN = res_FFNN.append({'layer1':units_1, 'layer2':units_2, 'layer3':units_3, 'epoch':ep+1,
                                  'trainScore':history['accuracy'][ep],
                                  'testScore':history['val_accuracy'][ep]}, ignore_index=True)
                
# get run time
end_time = time.time()
delta_time = end_time - start_time
print('\n')
print("total run time: {} mins".format(delta_time/60))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/3
2162/2162 [==============================] - 10s 4ms/step - loss: 0.5907 - accuracy: 0.6822 - val_loss: 0.5653 - val_accuracy: 0.6995
Epoch 2/3
2162/2162 [==============================] - 9s 4ms/step - loss: 0.5285 - accuracy: 0.7279 - val_loss: 0.5734 - val_accuracy: 0.6896
Epoch 3/3
2162/2162 [==============================] - 9s 4ms/step - loss: 0.4794 - accuracy: 0.7622 - val_loss: 0.6045 - val_accuracy: 0.6756




 50%|█████     | 1/2 [00:29<00:29, 29.21s/it]

Epoch 1/3
2162/2162 [==============================] - 10s 4ms/step - loss: 0.5837 - accuracy: 0.6851 - val_loss: 0.5620 - val_accuracy: 0.7009
Epoch 2/3
2162/2162 [==============================] - 9s 4ms/step - loss: 0.5236 - accuracy: 0.7274 - val_loss: 0.5745 - val_accuracy: 0.6906
Epoch 3/3
2162/2162 [==============================] - 9s 4ms/step - loss: 0.4734 - accuracy: 0.7574 - val_loss: 0.6212 - val_accuracy: 0.6828




100%|██████████| 2/2 [00:58<00:00, 29.20s/it]

 33%|███▎      | 1/3 [00:58<01:56, 58.41s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 1/3
2162/2162 [==============================] - 410s 189ms/step - loss: 0.5745 - accuracy: 0.6893 - val_loss: 0.5574 - val_accuracy: 0.7016
Epoch 2/3
2162/2162 [==============================] - 407s 188ms/step - loss: 0.5070 - accuracy: 0.7364 - val_loss: 0.5773 - val_accuracy: 0.6928
Epoch 3/3
2162/2162 [==============================] - 408s 189ms/step - loss: 0.4167 - accuracy: 0.7900 - val_loss: 0.6739 - val_accuracy: 0.6647




 50%|█████     | 1/2 [20:26<20:26, 1226.52s/it]

Epoch 1/3
2162/2162 [==============================] - 409s 189ms/step - loss: 0.5735 - accuracy: 0.6893 - val_loss: 0.5574 - val_accuracy: 0.7007
Epoch 2/3
2162/2162 [==============================] - 407s 188ms/step - loss: 0.5052 - accuracy: 0.7380 - val_loss: 0.5810 - val_accuracy: 0.6921
Epoch 3/3
2162/2162 [==============================] - 408s 188ms/step - loss: 0.4104 - accuracy: 0.7937 - val_loss: 0.6926 - val_accuracy: 0.6610




100%|██████████| 2/2 [40:51<00:00, 1225.56s/it]

100%|██████████| 4/4 [2:37:25<00:00, 2361.42s/it]



total run time: 157.42807729641598 mins


In [34]:
resLong = res_FFNN.melt(id_vars=['layer1', 'layer2', 'layer3', 'epoch'], value_vars=['trainScore', 'testScore'],
         var_name='group', value_name='score')

layer1_20 = alt.Chart(resLong[resLong.layer1==20]).mark_line().encode(
    row = 'layer2',
    column = 'layer3',
    x = 'epoch',
    y = alt.Y('score', scale=alt.Scale(domain=[0.6,0.8])),
    color = 'group',
    tooltip = 'score'
).properties(width=50, height=50)

layer1_100 = alt.Chart(resLong[resLong.layer1==100]).mark_line().encode(
    row = 'layer2',
    column = 'layer3',
    x = 'epoch',
    y = alt.Y('score', scale=alt.Scale(domain=[0.6,0.8])),
    color = 'group',
    tooltip = 'score'
).properties(width=50, height=50)

layer1_200 = alt.Chart(resLong[resLong.layer1==200]).mark_line().encode(
    row = 'layer2',
    column = 'layer3',
    x = 'epoch',
    y = alt.Y('score', scale=alt.Scale(domain=[0.6,0.8])),
    color = 'group',
    tooltip = 'score'
).properties(width=50, height=50)

layer1_500 = alt.Chart(resLong[resLong.layer1==500]).mark_line().encode(
    row = 'layer2',
    column = 'layer3',
    x = 'epoch',
    y = alt.Y('score', scale=alt.Scale(domain=[0.6,0.8])),
    color = 'group',
    tooltip = 'score'
).properties(width=50, height=50)

layer1_20 | layer1_100 | layer1_200 | layer1_500

alt.HConcatChart(...)

In [35]:
# generate report result

res_FFNN_report = pd.DataFrame(columns=['rand', 'trainScore_accuracy', 'trainScore_recall',
                           'testScore_accuracy', 'testScore_recall'])

for rand in [10, 20, 30, 40, 50]:
    X_train_, X_test_, y_train_, y_test_ = \
        train_test_split(X_train_tfidf, y_train_tfidf, test_size=0.2, random_state=rand)
    
    input_shape = X_train_.shape[1]
    
    inputs = keras.Input(shape=(input_shape, ))
    x = layers.Dense(20, activation='relu')(inputs)
    x = layers.Dense(50, activation='relu')(x)
    x = layers.Dense(20, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', metrics=['accuracy', 'Recall'], optimizer='adam')
    model.fit(x=X_train_, y=y_train_.values, validation_data=(X_test_, y_test_.values),
        epochs=2, batch_size=128, verbose=1)

    history = model.history.history

    res_FFNN_report = res_FFNN_report.append({'rand':rand, 'trainScore_accuracy':history['accuracy'][1],
                      'trainScore_recall':history['recall'][1],
                      'testScore_accuracy':history['val_accuracy'][1],
                      'testScore_recall':history['val_recall'][1],}, ignore_index=True)

Epoch 1/2
1730/1730 [==============================] - 9s 4ms/step - loss: 0.5869 - accuracy: 0.6830 - recall: 0.7100 - val_loss: 0.5653 - val_accuracy: 0.6964 - val_recall: 0.7195
Epoch 2/2
1730/1730 [==============================] - 8s 4ms/step - loss: 0.5188 - accuracy: 0.7333 - recall: 0.7536 - val_loss: 0.5785 - val_accuracy: 0.6873 - val_recall: 0.7003
Epoch 1/2
1730/1730 [==============================] - 9s 4ms/step - loss: 0.5856 - accuracy: 0.6844 - recall: 0.7098 - val_loss: 0.5702 - val_accuracy: 0.6931 - val_recall: 0.7603
Epoch 2/2
1730/1730 [==============================] - 7s 4ms/step - loss: 0.5185 - accuracy: 0.7333 - recall: 0.7586 - val_loss: 0.5870 - val_accuracy: 0.6855 - val_recall: 0.7096
Epoch 1/2
1730/1730 [==============================] - 9s 4ms/step - loss: 0.5875 - accuracy: 0.6839 - recall: 0.7060 - val_loss: 0.5688 - val_accuracy: 0.6948 - val_recall: 0.7130
Epoch 2/2
1730/1730 [==============================] - 8s 4ms/step - loss: 0.5210 - accuracy: 0

In [36]:
res.describe()

,n_estimators,leraning_rate,trainScore,testScore,learning_rate
count,12.000000,0.0,12.000000,12.000000,12.000000
mean,103.333333,NaN,0.637953,0.633609,2.777500
std,81.053667,NaN,0.073196,0.071450,4.374058
min,10.000000,NaN,0.522841,0.518786,0.010000
25%,10.000000,NaN,0.609008,0.608536,0.077500
50%,100.000000,NaN,0.658910,0.658937,0.550000
75%,200.000000,NaN,0.684124,0.681406,3.250000
max,200.000000,NaN,0.722125,0.699082,10.000000


### b) CNN

In [37]:
start_time = time.time()

layers1 = [32, 128]
layers2 = [32, 128]
layers3 = [32, 128]
dropRate = 0.2
convSize = 7
striSize = 3
input_shape = 500
max_features = 40000
embedding_dim = 128
ep_size = 2
batch_size = 1024

res_CNN = pd.DataFrame(columns=['layer1', 'layer2', 'layer3', 'epoch', 'trainScore', 'testScore'])

for units_1 in tqdm(layers1):
    for units_2 in tqdm(layers2):
        for units_3 in tqdm(layers3):
        
            inputs = keras.Input(shape=(input_shape,), dtype="int64")

            x = layers.Embedding(max_features, embedding_dim)(inputs)
            x = layers.Dropout(dropRate)(x)

            x = layers.Conv1D(units_1, convSize, padding="valid", activation="relu", strides=striSize)(x)
            x = layers.Conv1D(units_2, convSize, padding="valid", activation="relu", strides=striSize)(x)
            x = layers.GlobalMaxPooling1D()(x)

            x = layers.Dense(units_3, activation="relu")(x)
            x = layers.Dropout(dropRate)(x)

            outputs = layers.Dense(1, activation="sigmoid")(x)

            model = keras.Model(inputs=inputs, outputs=outputs)
            model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
            model.fit(x=X_train_indice, y=y_train_indice.values, validation_data=(X_test_indice, y_test_indice.values),
                      epochs=ep_size, batch_size=batch_size, verbose=1)

            history = model.history.history

            for ep in np.arange(ep_size):
                res_CNN = res_CNN.append({'layer1':units_1, 'layer2':units_2, 'layer3':units_3, 'epoch':ep+1,
                                  'trainScore':history['accuracy'][ep],
                                  'testScore':history['val_accuracy'][ep]}, ignore_index=True)
                
                
# get run time
end_time = time.time()
delta_time = end_time - start_time
print('\n')
print("total run time: {} mins".format(delta_time/60))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/2
326/326 [==============================] - 73s 221ms/step - loss: 0.6024 - accuracy: 0.6763 - val_loss: 0.5675 - val_accuracy: 0.6986
Epoch 2/2
326/326 [==============================] - 72s 220ms/step - loss: 0.5137 - accuracy: 0.7481 - val_loss: 0.5560 - val_accuracy: 0.7084




 50%|█████     | 1/2 [02:24<02:24, 144.74s/it]

Epoch 1/2
326/326 [==============================] - 73s 220ms/step - loss: 0.5973 - accuracy: 0.6768 - val_loss: 0.5604 - val_accuracy: 0.7051
Epoch 2/2
326/326 [==============================] - 71s 217ms/step - loss: 0.4954 - accuracy: 0.7577 - val_loss: 0.5565 - val_accuracy: 0.7097




100%|██████████| 2/2 [04:48<00:00, 144.09s/it]

 50%|█████     | 1/2 [04:48<04:48, 288.18s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 1/2
326/326 [==============================] - 91s 278ms/step - loss: 0.5920 - accuracy: 0.6813 - val_loss: 0.5548 - val_accuracy: 0.7063
Epoch 2/2
326/326 [==============================] - 91s 279ms/step - loss: 0.5029 - accuracy: 0.7477 - val_loss: 0.5514 - val_accuracy: 0.7108




 50%|█████     | 1/2 [03:02<03:02, 182.35s/it]

Epoch 1/2
326/326 [==============================] - 92s 278ms/step - loss: 0.5879 - accuracy: 0.6833 - val_loss: 0.5551 - val_accuracy: 0.7065
Epoch 2/2
326/326 [==============================] - 91s 280ms/step - loss: 0.4942 - accuracy: 0.7544 - val_loss: 0.5513 - val_accuracy: 0.7072




100%|██████████| 2/2 [06:05<00:00, 182.58s/it]

100%|██████████| 2/2 [21:37<00:00, 648.73s/it]



total run time: 21.624282042185467 mins


In [38]:
resLong = res_CNN.melt(id_vars=['layer1', 'layer2', 'layer3', 'epoch'], value_vars=['trainScore', 'testScore'],
         var_name='group', value_name='score')

layer1_32 = alt.Chart(resLong[resLong.layer1==32]).mark_line().encode(
    row = 'layer2',
    column = 'layer3',
    x = 'epoch',
    y = alt.Y('score', scale=alt.Scale(domain=[0.6,0.8])),
    color = 'group',
    tooltip = 'score'
).properties(width=50, height=50)

layer1_128 = alt.Chart(resLong[resLong.layer1==128]).mark_line().encode(
    row = 'layer2',
    column = 'layer3',
    x = 'epoch',
    y = alt.Y('score', scale=alt.Scale(domain=[0.6,0.8])),
    color = 'group',
    tooltip = 'score'
).properties(width=50, height=50)

layer1_32 | layer1_128

alt.HConcatChart(...)

In [39]:
# generate report result

res_CNN_report = pd.DataFrame(columns=['rand', 'trainScore_accuracy', 'trainScore_recall',
                           'testScore_accuracy', 'testScore_recall'])

for rand in [10, 20, 30, 40, 50]:
    X_train_, X_test_, y_train_, y_test_ = \
        train_test_split(X_train_indice.numpy(), y_train_indice, test_size=0.2, random_state=rand)
    
    input_shape = X_train_.shape[1]
    
    inputs = keras.Input(shape=(input_shape,), dtype="int64")

    x = layers.Embedding(40000, 128)(inputs)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
    x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
    x = layers.GlobalMaxPooling1D()(x)

    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', metrics=['accuracy', 'Recall'], optimizer='adam')
    model.fit(x=X_train_, y=y_train_.values, validation_data=(X_test_, y_test_.values),
              epochs=2, batch_size=1024, verbose=1)

    history = model.history.history
    
    res_CNN_report = res_CNN_report.append({'rand':rand, 'trainScore_accuracy':history['accuracy'][1],
                      'trainScore_recall':history['recall'][1],
                      'testScore_accuracy':history['val_accuracy'][1],
                      'testScore_recall':history['val_recall'][1],}, ignore_index=True)

Epoch 1/2
261/261 [==============================] - 74s 278ms/step - loss: 0.5959 - accuracy: 0.6778 - recall: 0.7028 - val_loss: 0.5638 - val_accuracy: 0.6994 - val_recall: 0.7564
Epoch 2/2
261/261 [==============================] - 72s 275ms/step - loss: 0.4972 - accuracy: 0.7564 - recall: 0.7578 - val_loss: 0.5626 - val_accuracy: 0.7022 - val_recall: 0.6953
Epoch 1/2
261/261 [==============================] - 74s 278ms/step - loss: 0.5947 - accuracy: 0.6787 - recall: 0.7087 - val_loss: 0.5624 - val_accuracy: 0.7010 - val_recall: 0.7684
Epoch 2/2
261/261 [==============================] - 72s 275ms/step - loss: 0.4976 - accuracy: 0.7547 - recall: 0.7515 - val_loss: 0.5682 - val_accuracy: 0.7038 - val_recall: 0.6471
Epoch 1/2
261/261 [==============================] - 74s 279ms/step - loss: 0.5963 - accuracy: 0.6774 - recall: 0.7045 - val_loss: 0.5620 - val_accuracy: 0.7018 - val_recall: 0.6997
Epoch 2/2
261/261 [==============================] - 72s 278ms/step - loss: 0.4992 - accur

In [ ]:
res_CNN_report.describe()

,rand,trainScore_accuracy,trainScore_recall,testScore_accuracy,testScore_recall
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,30.000000,0.754349,0.753173,0.702359,0.652430
std,15.811388,0.001627,0.002677,0.001847,0.030764
min,10.000000,0.752620,0.751083,0.699294,0.609005
25%,20.000000,0.752788,0.751455,0.702248,0.647069
50%,30.000000,0.754700,0.752717,0.702653,0.652369
75%,40.000000,0.755214,0.752851,0.703763,0.658440
max,50.000000,0.756421,0.757759,0.703838,0.695267


### c) LSTM

In [ ]:
start_time = time.time()

layers1 = [16, 64]
layers2 = [16, 64]
input_shape = 500
max_features = 40000
embedding_dim = 128
ep_size = 2
batch_size = 1024

res_LSTM = pd.DataFrame(columns=['layer1', 'layer2', 'epoch', 'trainScore', 'testScore'])

for units_1 in tqdm(layers1):
    for units_2 in tqdm(layers2):
            
        inputs = keras.Input(shape=(input_shape,), dtype="int32")
        
        x = layers.Embedding(max_features, embedding_dim)(inputs)
        x = layers.Bidirectional(layers.LSTM(units_1, return_sequences=True))(x)
        x = layers.Bidirectional(layers.LSTM(units_2))(x)

        outputs = layers.Dense(1, activation="sigmoid")(x)
        
        model = keras.Model(inputs, outputs)
        model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
        model.fit(x=X_train_indice, y=y_train_indice.values, validation_data=(X_test_indice, y_test_indice.values),
                      epochs=ep_size, batch_size=batch_size, verbose=1)

        history = model.history.history

        for ep in np.arange(ep_size):
            res_LSTM = res_LSTM.append({'layer1':units_1, 'layer2':units_2, 'epoch':ep+1,
                              'trainScore':history['accuracy'][ep],
                              'testScore':history['val_accuracy'][ep]}, ignore_index=True)
            
# get run time
end_time = time.time()
delta_time = end_time - start_time
print('\n')
print("total run time: {} mins".format(delta_time/60))

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/2
326/326 [==============================] - 368s 1s/step - loss: 0.5895 - accuracy: 0.6811 - val_loss: 0.5590 - val_accuracy: 0.7023
Epoch 2/2
326/326 [==============================] - 360s 1s/step - loss: 0.5228 - accuracy: 0.7292 - val_loss: 0.5608 - val_accuracy: 0.6996



 50%|█████     | 1/2 [12:08<12:08, 728.38s/it]

Epoch 1/2
326/326 [==============================] - 518s 2s/step - loss: 0.5901 - accuracy: 0.6785 - val_loss: 0.5605 - val_accuracy: 0.7006
Epoch 2/2
326/326 [==============================] - 512s 2s/step - loss: 0.5277 - accuracy: 0.7256 - val_loss: 0.5629 - val_accuracy: 0.7035



  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/2
326/326 [==============================] - 671s 2s/step - loss: 0.5887 - accuracy: 0.6815 - val_loss: 0.5610 - val_accuracy: 0.6978
Epoch 2/2
326/326 [==============================] - 667s 2s/step - loss: 0.5246 - accuracy: 0.7281 - val_loss: 0.5655 - val_accuracy: 0.6998



 50%|█████     | 1/2 [22:18<22:18, 1338.04s/it]

Epoch 1/2
326/326 [==============================] - 841s 3s/step - loss: 0.5869 - accuracy: 0.6823 - val_loss: 0.5595 - val_accuracy: 0.6989
Epoch 2/2
326/326 [==============================] - 835s 3s/step - loss: 0.5246 - accuracy: 0.7270 - val_loss: 0.5615 - val_accuracy: 0.7044



100%|██████████| 2/2 [1:19:34<00:00, 2387.05s/it]



total run time: 79.56831512451171 mins


In [ ]:
resLong = res_LSTM.melt(id_vars=['layer1', 'layer2', 'epoch'], value_vars=['trainScore', 'testScore'],
         var_name='group', value_name='score')

alt.Chart(resLong).mark_line().encode(
    row = 'layer1',
    column = 'layer2',
    x = 'epoch',
    y = alt.Y('score', scale=alt.Scale(domain=[0.65,0.75])),
    color = 'group',
    tooltip = 'score'
).properties(width=50, height=50)

alt.Chart(...)

In [ ]:
# generate report result

res_LSTM_report = pd.DataFrame(columns=['rand', 'trainScore_accuracy', 'trainScore_recall',
                           'testScore_accuracy', 'testScore_recall'])

for rand in [10, 20, 30, 40, 50]:
    X_train_, X_test_, y_train_, y_test_ = \
        train_test_split(X_train_indice.numpy(), y_train_indice, test_size=0.2, random_state=rand)
    
    input_shape = X_train_.shape[1]
    
    inputs = keras.Input(shape=(input_shape,), dtype="int64")

    x = layers.Embedding(40000, 128)(inputs)
    x = layers.Bidirectional(layers.LSTM(16, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(16))(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs, outputs)
    model.compile(loss='binary_crossentropy', metrics=['accuracy', 'Recall'], optimizer='adam')
    model.fit(x=X_train_indice, y=y_train_indice.values, validation_data=(X_test_indice, y_test_indice.values),
                  epochs=2, batch_size=1024, verbose=1)

    history = model.history.history
    
    res_LSTM_report = res_LSTM_report.append({'rand':rand, 'trainScore_accuracy':history['accuracy'][1],
                      'trainScore_recall':history['recall'][1],
                      'testScore_accuracy':history['val_accuracy'][1],
                      'testScore_recall':history['val_recall'][1],}, ignore_index=True)

Epoch 1/2
326/326 [==============================] - 369s 1s/step - loss: 0.5938 - accuracy: 0.6803 - recall: 0.7070 - val_loss: 0.5633 - val_accuracy: 0.7003 - val_recall: 0.7577
Epoch 2/2
326/326 [==============================] - 366s 1s/step - loss: 0.5280 - accuracy: 0.7285 - recall: 0.7399 - val_loss: 0.5628 - val_accuracy: 0.7010 - val_recall: 0.7406
Epoch 1/2
326/326 [==============================] - 373s 1s/step - loss: 0.5900 - accuracy: 0.6811 - recall: 0.7039 - val_loss: 0.5600 - val_accuracy: 0.7009 - val_recall: 0.7149
Epoch 2/2
326/326 [==============================] - 368s 1s/step - loss: 0.5223 - accuracy: 0.7295 - recall: 0.7366 - val_loss: 0.5587 - val_accuracy: 0.7012 - val_recall: 0.7009
Epoch 1/2
326/326 [==============================] - 376s 1s/step - loss: 0.5914 - accuracy: 0.6803 - recall: 0.6971 - val_loss: 0.5599 - val_accuracy: 0.7014 - val_recall: 0.7119
Epoch 2/2
326/326 [==============================] - 370s 1s/step - loss: 0.5246 - accuracy: 0.7283 

In [ ]:
res_LSTM_report.describe()

,rand,trainScore_accuracy,trainScore_recall,testScore_accuracy,testScore_recall
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,30.000000,0.728206,0.738150,0.702303,0.721706
std,15.811388,0.001119,0.002524,0.001251,0.021069
min,10.000000,0.726397,0.734531,0.701034,0.700940
25%,20.000000,0.728290,0.736631,0.701154,0.706265
50%,30.000000,0.728365,0.739031,0.702510,0.713052
75%,40.000000,0.728503,0.739924,0.702762,0.740610
max,50.000000,0.729477,0.740632,0.704057,0.747662


### d) Transformer

In [ ]:
## Implement a transformer blocker

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
## Implement embedding layer
## two layers, one for token, one for positions

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
embed_dim_list = [32]  # Embedding size for each token
num_heads_list = [2]  # Number of attention heads
ff_dim_list = [32]  # Hidden layer size in feed forward network inside transformer
maxlen = 500
vocab_size = 40000
ep_size = 3
batch_size = 1024

res = pd.DataFrame(columns=['embed_dim', 'num_heads', 'ff_dim',
                            'epoch', 'trainScore', 'testScore'])

for embed_dim in tqdm(embed_dim_list):
    for num_heads in tqdm(num_heads_list):
        for ff_dim in tqdm(ff_dim_list):

            inputs = layers.Input(shape=(maxlen,))

            embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
            x = embedding_layer(inputs)
            transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
            x = transformer_block(x)
            x = layers.GlobalAveragePooling1D()(x)
            x = layers.Dropout(0.1)(x)
            x = layers.Dense(20, activation="relu")(x)
            x = layers.Dropout(0.1)(x)

            outputs = layers.Dense(1, activation="sigmoid")(x)

            model = keras.Model(inputs=inputs, outputs=outputs)
            model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                          optimizer='adam')
            model.fit(x=X_train_indice, y=y_train_indice.values,
                      validation_data=(X_test_indice, y_test_indice.values),
                      epochs=ep_size, batch_size=batch_size, verbose=1)

            history = model.history.history

            for ep in np.arange(ep_size):
                res = res.append({'embed_dim':embed_dim, 'num_heads':num_heads,
                                  'ff_dim':ff_dim,'epoch':ep+1,
                                  'trainScore':history['accuracy'][ep],
                                  'testScore':history['val_accuracy'][ep]},
                                  ignore_index=True)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/3
326/326 [==============================] - 1256s 4s/step - loss: 0.6359 - accuracy: 0.6319 - val_loss: 0.5828 - val_accuracy: 0.6938
Epoch 2/3
326/326 [==============================] - 1278s 4s/step - loss: 0.5637 - accuracy: 0.7112 - val_loss: 0.5691 - val_accuracy: 0.7009
Epoch 3/3
326/326 [==============================] - 1279s 4s/step - loss: 0.5340 - accuracy: 0.7289 - val_loss: 0.5686 - val_accuracy: 0.7014




100%|██████████| 1/1 [1:03:32<00:00, 3812.98s/it]

100%|██████████| 1/1 [1:03:32<00:00, 3812.98s/it]
